<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [2]:
import csv
from collections import namedtuple   # Convenient to store the data rows

DATA_FILE = './data/chipotle.tsv'

data=open(DATA_FILE, 'r')
dt=csv.reader(data, delimiter='\t')  # indicate that it's a TSV file

file_nested_list=[]
for row in dt:  
    file_nested_list.append(row)


### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [3]:
header = file_nested_list[0]

In [4]:
data = file_nested_list[1:]

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [6]:
sorted_data = sorted(data, key=lambda x: int(x[0]))

In [16]:
from re import sub
from decimal import Decimal

average_price = 0
total_order = 0
total_price = 0
items_per_order=0
order_price=0
row=0
prev_order_id=''

for order in sorted_data:
    row += 1
    if order[0] != prev_order_id:
        if row > 1:
            total_order += 1
            total_price += order_price  # include previous order price into total_price
            
            order_price = 0  # re-initialise values per order 
            items_per_order = 0
            
    item_price = round(Decimal(sub(r'[^\d\-.]', '', order[4])),2)  # extract the decimal value price of each item
    order_price += item_price  # accumulate price per order... quantity not relevant as item_price incorporates the quantity
    prev_order_id = order[0]  # save order_id for comparison of each row

if row != 0:
    # include previous order price into total_price
    total_order += 1
    total_price += order_price
    average_price = round(total_price / total_order,2)
    
    print ('Average price per order: $' + str(average_price))

Average price per order: $18.81


### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [8]:
unique_sodas = set([])

for order in sorted_data:
    if order[2] in ('Canned Soda', 'Canned Soft Drink'):
        unique_sodas.add(order[3])
        
unique_sodas

{'[Coca Cola]',
 '[Coke]',
 '[Diet Coke]',
 '[Diet Dr. Pepper]',
 '[Dr. Pepper]',
 '[Lemonade]',
 '[Mountain Dew]',
 '[Nestea]',
 '[Sprite]'}

---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [9]:
# Initialise number of Burrito orders and the total toppings
burrito_count = 0
toppings_num = 0

for order in sorted_data:
    if 'Burrito' in order[2]:    # Search for the word Burrito in the item_name, assuming the case used is exactly this
        burrito_count += 1
        toppings_num += len(data[9][3].split(','))   #  Split the choice_description string into a list, based on comma

print('Average number of toppings per burito : ', int(toppings_num/burrito_count))


Average number of toppings per burito :  7


### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [12]:
sorted_by_item_name = sorted(data, key=lambda x: x[2])

In [15]:
# Initialise variables
from collections import defaultdict
dict_chips = defaultdict(list)
chips_count = 0
chips_items = 0
prev_item_name=''

for order in sorted_by_item_name:
    if 'Chip' in order[2]:
        chips_items += 1
        if (order[2] != prev_item_name) and (chips_items > 1):
            dict_chips[prev_item_name].append(chips_count)  # append entry into dictionary
            chips_count = 0
            
        chips_count += int(order[1])  # increment total order by the quantity
        prev_item_name = order[2]  # save item_name for comparison of each row

if chips_items != 0 and chips_count != 0:  # if the latest entry have not been included yet
    dict_chips[prev_item_name].append(chips_count)  # append entry into dictionary
    
dict_chips


defaultdict(list,
            {'Chips': [230],
             'Chips and Fresh Tomato Salsa': [130],
             'Chips and Guacamole': [506],
             'Chips and Mild Fresh Tomato Salsa': [1],
             'Chips and Roasted Chili Corn Salsa': [23],
             'Chips and Roasted Chili-Corn Salsa': [18],
             'Chips and Tomatillo Green Chili Salsa': [45],
             'Chips and Tomatillo Red Chili Salsa': [50],
             'Chips and Tomatillo-Green Chili Salsa': [33],
             'Chips and Tomatillo-Red Chili Salsa': [25],
             'Side of Chips': [110]})

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!
